In [1]:
import struct
from os import listdir
from os.path import exists, getsize, join, isfile
import csv
import progressbar

channels = {b'B':"Battery Current (A)", b'P': "Air Pressure (P)", b'H':"Relative Humidity (percent)", b'M':"Battery Temperature (C)", b'N':"Internal Temperature (C)", b'T':"External Temperature (C)", b'1':"X Acceleration (g)", b'2':"Y Acceleration (g)", b'3':"Z Acceleration (g)", b'7':"X Mangetometer (uT)", b'8':"Y Mangetometer (uT)", b'9':"Z Mangetometer (uT)", b'4':"X Gyro (deg per sec)", b'5':"Y Gyro (deg per sec)", b'6':"Z Gyro (deg per sec)", b'L':"Latitude", b'O':"Longitude", b'A':"Altitude (m)", b'S':"Speed (m per s)", b'I':"Time (seconds)", b'V':"VOC density (ppb)", b'C':"eCO2 density (ppm)"} 

# Try to read the files until we get something valid
targetfiles = None
while targetfiles == None:
    try:
        print("file path?")
        filepath = input();
        print("file base (e.g. type \"LOG\" if the filenames are \"LOG1\", \"LOG2\" etc.)?")
        filename = input();
        allfiles = [f for f in listdir(filepath) if isfile(join(filepath, f))]
        targetfiles = [f for f in allfiles if f[0:len(filename)] == filename]
        break
    except:
        print("Invalid Path")
        continue
    
    if len(targetfiles) == 0:
        print("No files with this name!")
        targetfiles = None

print("Files:")
print(targetfiles)

file path?
94001
file base (e.g. type "LOG" if the filenames are "LOG1", "LOG2" etc.)?
LOG
Files:
['LOG0', 'LOG1']


In [2]:
# Calculate some values for the loop
n_cycles = int(sum([getsize(join(filepath, targetfile))/9 for targetfile in targetfiles]))
current_cycle = 1
file_idx = 0

with progressbar.ProgressBar(max_value=n_cycles) as bar:
    for tf in targetfiles: 
        try:
            targetfile = filename + str(file_idx)
            print("Opening " + targetfile)
            file = open(join(filepath, targetfile), "rb")
        except:
            print("File " + targetfile + " DNE")
            file_idx += 1
            break;

        dataIn = file.read(1)
        while dataIn:
            try: # Check if the dataIn is a valid data source
                title = channels[dataIn]
            except: # If not, go to the next byte
                print("Invalid Channel " + str(dataIn))
                dataIn = file.read(1)
                continue

            try: # Try to read the data in to process
                byteLong  = file.read(4)
                byteFloat = file.read(4)
                # Convert byteLong and byteFloat to an unsigned long and float respectively
                [timestamp] = struct.unpack("L", byteLong)
                [data] = struct.unpack("f", byteFloat)
            except: # If it fails, we have probably reached EOF; break out of reading this file
                print("Failed to read bytes")
                dataIn = file.read(1)
                break

            # Create CSV file name
            csvfilename = join(filepath, title + ".csv")
            # Check if it already exists
            # If it doesn't exist, create the file and write the header
            if (not exists(csvfilename)):
                print("New file: " + csvfilename)
                f = open(csvfilename, "a")
                f.write("Time (ms), " + title + "\n")
                f.close()

            # Now that we know it exists and has a header, open it and write the data
            with open(csvfilename, 'a', newline='', encoding='UTF8') as f:
                writer = csv.writer(f)
                row = [str(timestamp), str(data)]
                writer.writerow(row)

            # Update the progress bar
            current_cycle += 1
            current_cycle = min(current_cycle, n_cycles) # prevent rounding errors from causing problems
            bar.update(current_cycle)
            # Read next byte
            dataIn = file.read(1)
        # Go to the next file
        file_idx += 1

print("done")

  0% (369 of 116154) |                   | Elapsed Time: 0:00:00 ETA:  00:00:00

Opening LOG0
New file: 94001\Battery Current (A).csv
New file: 94001\Air Pressure (P).csv
New file: 94001\Relative Humidity (percent).csv
New file: 94001\Battery Temperature (C).csv
New file: 94001\Internal Temperature (C).csv
New file: 94001\External Temperature (C).csv
New file: 94001\X Acceleration (g).csv
New file: 94001\Y Acceleration (g).csv
New file: 94001\Z Acceleration (g).csv
New file: 94001\X Gyro (deg per sec).csv
New file: 94001\Y Gyro (deg per sec).csv
New file: 94001\Z Gyro (deg per sec).csv
New file: 94001\X Mangetometer (uT).csv
New file: 94001\Y Mangetometer (uT).csv
New file: 94001\Z Mangetometer (uT).csv
New file: 94001\Latitude.csv
New file: 94001\Longitude.csv
New file: 94001\Altitude (m).csv
New file: 94001\Speed (m per s).csv
New file: 94001\Time (seconds).csv
New file: 94001\VOC density (ppb).csv
New file: 94001\eCO2 density (ppm).csv


 57% (66605 of 116154) |#########        | Elapsed Time: 0:00:14 ETA:   0:00:10

Opening LOG1


100% (116154 of 116154) |################| Elapsed Time: 0:00:25 Time:  0:00:25


done
